<a href="https://colab.research.google.com/github/jmwangi20/The-beginning-/blob/main/Fake_news_prediction_using_machine_learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we are going to go through an end to end problem to classify whether a news article is either real or fake

**Importing the dependencies**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as  plt
%matplotlib inline
# re is used to search for words in a sentence or a paragraph...the short form for regular expressions 
import re
# stop words are those words which do not add alot of meaning in a paragraph or a sentence
from nltk.corpus import stopwords
# PorterStemmer is used to stem the suffix and the prefix in words to the root form eg from breaking to break
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# Linear model because it perfoms well on classification tasks
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
#check the length of the stopwords
print(len(stopwords.words("english")))

179


**Data_Preprocessing**

The data has been provided to as by kaggle https://www.kaggle.com/c/fake-news/data?select=train.csv



In [16]:
# Loading the dataset into a pandas dataframe
news_dataset=pd.read_csv("/content/train.csv",
                 engine="python",
                 error_bad_lines=False,  
                 encoding='utf-8')
news_dataset.shape

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 19793, saw 12
Skipping line 19794: Expected 5 fields 

(20822, 5)

In [17]:
news_dataset.shape

(20822, 5)

In [18]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
# check the lenght of the dataframe
len(news_dataset)

20822

In [20]:
news_dataset.tail()

,id,title,author,text,label
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20821,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [26]:
news_dataset.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [27]:
news_dataset.describe()

,id,title,author,text,label
count,20822,20257,18847,20763,20800
unique,20822,19817,4205,20388,4
top,0,The Dark Agenda Behind Globalism And Open Borders,Pam Key,,1
freq,1,5,243,75,10411


In [28]:
news_dataset["label"].value_counts()

1                                                   10411
0                                                   10387
 как люди воспринимают своё положение.                  1
 чтобы это была дорога с двусторонним движением.        1
Name: label, dtype: int64

In [30]:
# fill the missing values with a null string
news_dataset=news_dataset.fillna("")

In [31]:
# Merging the author column and the news title
news_dataset["content"]=news_dataset["author"]+" "+news_dataset["title"]


In [32]:
news_dataset["content"]

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20817    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20818    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20819    Michael J. de la Merced and Rachel Abrams Macy...
20820    Alex Ansary NATO, Russia To Hold Parallel Exer...
20821              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20822, dtype: object

In [33]:
# Separating the data and the labels
X=news_dataset.drop("label",axis=1)
y=news_dataset["label"]


In [34]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20817  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20818  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20819  20797  Macy’s Is Said to Receive Takeover Approach by...   
20820  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20821  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [35]:
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20817    0
20818    0
20819    0
20820    1
20821    1
Name: label, Length: 20822, dtype: object


In [37]:
port_stem=PorterStemmer()

**Stemming:**Stemming is the process of reducing a wod to its root word

eg from breaking to break

In [38]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=" ".join(stemmed_content)
  return stemmed_content
  


In [39]:
# apply the function to the content
news_dataset["content"]=news_dataset["content"].apply(stemming)

In [40]:
print(news_dataset["content"])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20817    jerom hudson rapper trump poster child white s...
20818    benjamin hoffman n f l playoff schedul matchup...
20819    michael j de la merc rachel abram maci said re...
20820    alex ansari nato russia hold parallel exercis ...
20821                            david swanson keep f aliv
Name: content, Length: 20822, dtype: object


In [42]:
# Separating the data and the labels
X=news_dataset["content"].values
y=news_dataset["label"].values

In [43]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [44]:
print(y)

['1' '0' '1' ... '0' '1' '1']


In [45]:
# check the shape of the dataset
X.shape,y.shape

((20822,), (20822,))

In [47]:
# converting Textual data into numerical
vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(X)


In [48]:
print(X)

  (0, 15695)	0.2848972510323515
  (0, 2484)	0.3676990592743697
  (0, 7695)	0.24789859828097843
  (0, 8633)	0.29217179802324905
  (0, 2960)	0.24689141014750257
  (0, 13478)	0.2560149172431211
  (0, 4975)	0.2333632865672138
  (0, 267)	0.27014778053672167
  (0, 3794)	0.2705797813233836
  (0, 7008)	0.21878792690522433
  (0, 8912)	0.36364344795259246
  (0, 3602)	0.35994096489813754
  (1, 1894)	0.15524631265972175
  (1, 2224)	0.3827200188088641
  (1, 16807)	0.30071934786077886
  (1, 1497)	0.2939921887587299
  (1, 2814)	0.19096625123288233
  (1, 6819)	0.1904866118084521
  (1, 5506)	0.7143145691090199
  (1, 3570)	0.2637458518155929
  (2, 5392)	0.3866575731579265
  (2, 5971)	0.3474726097377811
  (2, 9623)	0.4935010343775579
  (2, 15620)	0.4154491828226571
  (2, 2944)	0.3180050294894952
  :	:
  (20819, 1287)	0.33537027441279527
  (20819, 13126)	0.24825540550304984
  (20819, 12348)	0.2726336911024517
  (20819, 14975)	0.311587804384769
  (20819, 12142)	0.2477854180074755
  (20819, 9521)	0.29541609

In [49]:
# splitting the data into training and testing data
X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                               test_size=0.2,
                                               random_state=2)


In [51]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((16657, 17136), (16657,), (4165, 17136), (4165,))

**Training the model**

In [52]:
model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

## Checking the accuracy of the training data


In [57]:
X_train_prediction=model.predict(X_train)
training_accuracy=accuracy_score(y_train,X_train_prediction)
print(f"The training accuracy is:{training_accuracy*100:.2f}")

The training accuracy is:99.24


In [55]:
X_train.shape

(16657, 17136)

In [56]:
X_train_prediction.shape

(16657,)

In [58]:
#check the test predictions
test_predictions=model.predict(X_test)
test_score=accuracy_score(y_test,test_predictions)
print(f"The test score is :{test_score*100:.2f}")

The test score is :98.15
